## STACKS+BOWTIE+BLAST Pipeline for Population Genomics Analysis

This is the full pipeline that I'm using to analyze my Pacific cod time series data for FISH546. Our lab uses the [Stacks](https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=stacks%20catchen) pipeline, followed by filtering our catalog with Bowtie and BLAST to filter down our catalog of loci into a de novo reference genome, and then rerunning the Stacks pipeline. My lab mate [Mary Fisher](https://github.com/mfisher5) made a great diagram of the flow of steps in our lab's pipeline.

![image](https://github.com/mfisher5/mf-fish546-2016/raw/master/Diagrams/flowchart1.png)


### Steps

#### 1. Add library identifier to file name (optional)
When combining sequence data from different sequencing lanes but with redundant barcodes, I add an identifier for the library number (or lane number) so that I can process all of the files together.

#### 2. ``ustacks``
``ustacks`` reads fastq files and makes "stacks" of identical reads, which can be used to locate tags call SNPs within individuals.

#### 3.  ``cstacks``
``cstacks`` uses a subset of your individuals to make a catalog of loci that you will use to genotype all of your individuals. 

#### 4. ``sstacks``
``sstacks`` uses the catalog made in ``cstacks`` to genotype your individuals.

#### 5. ``populations``
``populations`` assigns individuals to populations and produces a genepop file and other summary statistics files.

#### 6. Filter catalog with ``bowtie``
We use ``bowtie`` align our catalog to itself. This filters out loci that align to different sequences in addition to aligning to themselves. This is supposed to remove highly repetitive tags that could interfere with our analysis.

#### 7. Filter catalog with ``BLAST``
We ``BLAST`` our catalog to itself to filter out more loci. This is supposed to remove highly repetitive tags that could interfere with our analysis.

#### 8. Create final reference genome and align reads with ``bowtie``
We use ``bowtie`` for a second time to align all of our reads to our filtered catalog, which is the input for ``pstacks``.

#### 9. ``pstacks``
``Pstacks`` works like ``ustacks``, except instead of aligning reads to themselves, it identifies stacks in the alignment file. It uses these stacks to find tags and call SNPs within inviduals.

#### 10. ``cstacks``
This second round of ``cstacks`` uses the ``pstacks`` output files to make a better catalog, as opposed to using one where we clustered de novo.

#### 11. ``sstacks``
This second round of ``sstacks`` genotypes indiviuals using the new ``cstacks`` catalog.

#### 12. ``populations``
``populations`` assigns individuals to populations and produces a genepop file and other summary statistics files. This second round of ``populations`` is much cleaner and more conservative.

#### 13. Additional filtering with Marine's scripts
Our postdoc Marine made additional filtering scripts for after the second ``populations`` to 

#### 14. Statistial tests in R


<br>
<br>
<br>

#### Go to working directory

In [1]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data/ 

[Errno 2] No such file or directory: '/Volumes/Time Machine Backups/Cod-Time-Series-Data/'
/Users/natalielowell/Git-repos/FISH546/Cod-Time-Series-Project/Notebooks


### 1. Adding library identifier to file names

If you are analyzing data run on multiple lanes, it may be useful to rename your files such that they have the unique library identifier (eg., \_L1 or \_L2) because barcodes will be redundant between libraries. I wrote a [script](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/add_lib_to_filename.py) that will add this to your filenames.



In [ ]:
!python add_lib_to_filename.py process_radtags_out/cod_lib1 _L1

In [ ]:
!python add_lib_to_filename.py process_radtags_out/cod_lib2 _L2

### 2. Running ``ustacks``

``ustacks`` [documentation](http://catchenlab.life.illinois.edu/stacks/comp/ustacks.php) highlights:

<br>ustacks -t file_type -f file_path [-d] [-r] [-o path] [-i id] [-m min_cov] [-M max_dist] [-p num_threads] [-R] [-H] [-h]
<br>t — input file Type. Supported types: fasta, fastq, gzfasta, or gzfastq.
<br>f — input file path.
<br>o — output path to write results.
<br>i — SQL ID to insert into the output to identify this sample.
<br>m — Minimum depth of coverage required to create a stack (default 2).
<br>M — Maximum distance (in nucleotides) allowed between stacks (default 2).
<br>N — Maximum distance allowed to align secondary reads to primary stacks (default: M + 2).
<br>R — retain unused reads.
<br>H — disable calling haplotypes from secondary reads.
<br>p — enable parallel execution with num_threads threads.
<br>h — display this help messsage.

<br>
<br>
Running custom python [script for ``ustacks``](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/pypipe_ustacks.py):

In [ ]:
!python pypipe_ustacks.py barcodes_samplenames.txt ./process_radtags_out ./ustacks_out

### 3. Running ``cstacks``
``cstacks`` [documentation](http://catchenlab.life.illinois.edu/stacks/comp/cstacks.php) highlights:

<br>cstacks -b batch_id -s sample_file [-s sample_file_2 ...] [-o path] [-n num] [-g] [-p num_threads] [--catalog path] [-h]
<br>p — enable parallel execution with num_threads threads.
<br>b — MySQL ID of this batch.
<br>s — TSV file from which to load radtags.
<br>o — output path to write results.
<br>m — include tags in the catalog that match to more than one entry.
<br>n — number of mismatches allowed between sample tags when generating the catalog.
<br>g — base catalog matching on genomic location, not sequence identity.
<br>h — display this help messsage.

<br>
<br>
Running custom python [script for ``cstacks``](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/pypipe_cstacks.py):


In [1]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data

/Volumes/Time Machine Backups/Cod-Time-Series-Data


In [26]:
!python pypipe_cstacks.py prt_out_filenames.txt stacks_b3 10 3 stacks_b3 3 5

cstacks paramters selected:
  Loci matched based on sequence identity.
  Number of mismatches allowed between stacks: 3
  Gapped alignments: disabled
Constructing catalog from 10 samples.
Initializing new catalog...
  Parsing stacks_b3/2015_101_1.tags.tsv.gz
  Parsing stacks_b3/2015_101_1.snps.tsv.gz
  Parsing stacks_b3/2015_101_1.alleles.tsv.gz
  37987 loci were newly added to the catalog.
Processing sample stacks_b3/2015_101_1 [2 of 10]
  Parsing stacks_b3/2005_464_1.tags.tsv.gz
  Parsing stacks_b3/2005_464_1.snps.tsv.gz
  Parsing stacks_b3/2005_464_1.alleles.tsv.gz
Searching for sequence matches...
  Distance allowed between stacks: 3; searching with a k-mer length of 35 (110 k-mers per read); 5 k-mer hits required.
  37987 loci in the catalog, 3925472 kmers in the catalog hash.
Merging matches into catalog...
  34143 loci were matched to a catalog locus.
  0 loci were matched to a catalog locus using gapped alignments.
  4583 loci were newly added to the catalog.
  185 loci matched

### 4. Running ``sstacks``
``sstacks`` [documentation](http://catchenlab.life.illinois.edu/stacks/comp/sstacks.php) highlights:

<br>sstacks -b batch_id -c catalog_file -s sample_file [-s sample_file_2 ...] [-o path] [-p num_threads] [-g] [-x] [-v] [-h]
<br>p — enable parallel execution with num_threads threads.
<br>b — MySQL ID of this batch.
<br>c — TSV file from which to load the catalog loci.
<br>s — TSV file from which to load sample loci.
<br>o — output path to write results.
<br>g — base matching on genomic location, not sequence identity.
<br>x — don’t verify haplotype of matching locus.
<br>v — print program version.
<br>h — display this help messsage.

Running custom python [script for ``sstacks``](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/pypipe_sstacks.py):


<br>WHEN RUNNING THIS SCRIPT, YOUR INPUTS AT THE COMMAND LINE ARE:
<br>python  
<br>{0}[pipeline filename] 
<br>{1}[files that came out of process rad tags]
<br>{2}[batch ID number]
<br>{3}[filepath to directory with catalog filename without file extension]
<br>{4}[filepath to directory w ustacks output files per sample]
<br>{5}[number of threads to use]

In [28]:
!python pypipe_sstacks.py prt_out_filenames.txt 3 stacks_b3/batch_3 stacks_b3 10 -o stacks_b3

['2005_297_1', '2005_298_1', '2005_299_1', '2005_384_1', '2005_385_1', '2005_387_1', '2005_388_1', '2005_389_1', '2005_457_1', '2005_459_1', '2005_460_1', '2005_461_1', '2005_462_1', '2005_463_1', '2005_464_1', '2005_465_1', '2005_466_1', '2005_467_1', '2005_495_1', '2005_559_1', '2009_123_1', '2009_124_1', '2009_127_1', '2009_128_1', '2009_129_1', '2009_131_1', '2009_135_1', '2009_141_1', '2009_147_1', '2009_164_1', '2009_169_1', '2009_170_1', '2009_171_1', '2009_172_1', '2009_173_1', '2009_174_1', '2009_176_1', '2009_178_1', '2009_532_1', '2009_536_1', '2009_537_1', '2010_114_1', '2010_115_1', '2010_116_1', '2010_117_1', '2010_118_1', '2010_119_1', '2010_123_1', '2010_127_1', '2010_184_1', '2010_185_1', '2010_187_1', '2010_188_1', '2010_189_1', '2010_211_1', '2010_212_1', '2010_213_1', '2010_215_1', '2010_216_1', '2010_218_1', '2010_219_1', '2010_220_1', '2010_221_1', '2010_222_1', '2010_240_1', '2010_242_1', '2010_243_1', '2010_244_1', '2010_246_1', '2010_247_1', '2010_250_1', '2010

In [3]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data/scripts

/Volumes/Time Machine Backups/Cod-Time-Series-Data/scripts


### 5. Running ``populations``

Documentation [here](http://catchenlab.life.illinois.edu/stacks/comp/populations.php).

Example code:
```
!populations -b 2 -P ustacks_out -M popmap1.txt -t 10 -r 0.50 -p 2 -m 5 --genepop
```

In [ ]:
!populations -b 3 -P test_20k_cutoff -M popmap_b3.txt -t 10 -r 0.50 -p 1 -m 10 --genepop

Counting retained loci after ``populations``:

In [4]:
!python count_unique_loci_gp.py ../test_20k_cutoff/batch_3_20k.genepop

15365


I retained 15,365 loci. See Jupyter Notebook "Solving low retained loci problem" to see how I ended up using this batch of samples (only samples with at least 20k retained loci after ``ustacks``).

### 6. Filter with ``Bowtie``

<br>
#### A. Make a fasta file for ``Bowtie`` 

Make a fasta file for ``Bowtie`` by running a custom python [script](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/genBOWTIEfasta.py) that Mary wrote, which requires (1) a text file with the header from the genepop file and (2) unzipped batch.catalog.tags file.

[Bowtie Documentation](http://bowtie-bio.sourceforge.net/manual.shtml) highlights:

<br>bowtie-build [options]
<br>Main arguments

reference_in
A comma-separated list of FASTA files containing the reference sequences to be aligned to, or, if -c is specified, the sequences themselves. E.g., <reference_in> might be chr1.fa,chr2.fa,chrX.fa,chrY.fa, or, if -c is specified, this might be <br>GGTCATCCT,ACGGGTCGT,CCGTTCTATGCGGCTTA.

-f
The reference input files (specified as (reference_in)) are FASTA files (usually having extension .fa, .mfa, .fna or similar).

ebwt_base
The basename of the index files to write. By default, bowtie-build writes files named <br>NAME.1.ebwt, NAME.2.ebwt, NAME.3.ebwt, NAME.4.ebwt, NAME.rev.1.ebwt, and NAME.rev.2.ebwt, <br>where NAME is <ebwt_base>.
<br>
<br>
<br>bowtie [options]
<br>Main arguments

-v (integer)
Report alignments with at most (integer) mismatches. -e and -l options are ignored and quality values have no effect on what alignments are valid. -v is mutually exclusive with -n.

-S/--sam
Print alignments in SAM format. See the SAM output section of the manual for details. To suppress all SAM headers, use --sam-nohead in addition to -S/--sam.

<br>
Here, I will make batch_3_loci.txt for this round by manually opening the genepop file in Textwrangler, and cutting and pasting the header line with the tag names into its own text file.

In [5]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data/test_20k_cutoff

/Volumes/Time Machine Backups/Cod-Time-Series-Data/test_20k_cutoff


Unzip the .catalog.tags file, which you will use as an argument to make the Bowtie FASTA:

In [6]:
!gzip -d batch_3.catalog.tags.tsv.gz

batch_3.catalog.tags.tsv already exists -- do you wish to overwrite (y or n)? ^C


Run script to make Bowtie FASTA, with first argument being text file you made with header from genepop, and second argument being the unzipped .catalog.tags file:

In [14]:
cd ../../scripts

/Volumes/Time Machine Backups/Cod-Time-Series-Data/scripts


In [15]:
!python genBOWTIEfasta.py ../test_20k_cutoff/batch_3_loci.txt ../test_20k_cutoff/batch_3.catalog.tags.tsv

Make a directory for ``Bowtie`` files and navigate there. Store the software there. I haven't figured out how to run ``Bowtie`` from different directories, so I moved all of the associated output files from previous batches into their own directory, so that I don't get confused when I make my new output files. 

Then I will use ``Bowtie`` to make a reference index de novo. Also, genBOWTIEfasta.py stores the new file in the same folder you're running the script from, so it may be worth editing the script at some point to direct where it saves. 

In [33]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data

/Volumes/Time Machine Backups/Cod-Time-Series-Data


In [8]:
!mkdir Bowtie

In [17]:
cd /Volumes/Time Machine Backups/Cod-Time-Series-Data/Bowtie/bowtie-1.1.2

/Volumes/Time Machine Backups/Cod-Time-Series-Data/Bowtie/bowtie-1.1.2


Make the Bowtie index, which produces some files that are all you need to align to the index.

In [18]:
!./bowtie-build seqsforBOWTIE.fa batch_3

Settings:
  Output files: "batch_3.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  seqsforBOWTIE.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 550152
Using parameters --bmax 412614 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 412614 --dcv 1024
Constructing suffix-array element generator
Building

Then, align it to itself and filter out any sequences that aligned to sequences other than themselves using a custom [script](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/parseBowtie_DD.py) that Dan wrote in our lab.

In [19]:
!./bowtie -f -v 3 --sam --sam-nohead \
batch_3 \
seqsforBOWTIE.fa \
batch_3_BOWTIEout.sam

# reads processed: 15282
# reads with at least one reported alignment: 15282 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 15282 alignments to 1 output stream(s)


In [20]:
cd ../../scripts

/Volumes/Time Machine Backups/Cod-Time-Series-Data/scripts


In [22]:
!python parseBowtie_DD.py ../Bowtie/bowtie-1.1.2/batch_3_BOWTIEout.sam ../Bowtie/bowtie-1.1.2/batch_3_BOWTIEout_filtered.fa

Number of Bowtie output lines read: 15282
Number of sequences written to output: 15282


``Bowtie`` filtering step filtered out 83 loci (15365 - 15282).

### 7. Filter with ``BLAST``

<br>
Change directory to highest project directory, here Cod Time Series Data. Then, make a directory for Blast (if you haven't done so already) and make a Blast database out of the output from Bowtie. This requires me to move the filtered fasta file, which I did manually. 

I have figured out how to run Blast from anywhere, so I'll make a new directory for this batch and move the Bowtie filetered fasta file there.

Then, we'll be filtering out any loci that match other loci equally well or better than to themselves, which is supposed to remove highly repetitive loci like microsatellites that can interfere with our data analysis. This [script](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/checkBlastResults_DD.py) for Blast filtering was also written by Dan.

In [23]:
cd ..

/Volumes/Time Machine Backups/Cod-Time-Series-Data


In [24]:
mkdir Blast_b3_20k

In [25]:
cd Blast_b3_20k

/Volumes/Time Machine Backups/Cod-Time-Series-Data/Blast_b3_20k


Make ``BLAST`` database:

In [26]:
!makeblastdb -in batch_3_BOWTIEout_filtered.fa \
-parse_seqids \
-dbtype nucl \
-out batch_3_BOWTIEfiltered



Building a new DB, current time: 12/12/2016 13:38:08
New DB name:   /Volumes/Time Machine Backups/Cod-Time-Series-Data/Blast_b3_20k/batch_3_BOWTIEfiltered
New DB title:  batch_3_BOWTIEout_filtered.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 15282 sequences in 0.735458 seconds.


``BLAST`` against itself:

In [27]:
!blastn -query batch_3_BOWTIEout_filtered.fa \
-db batch_3_BOWTIEfiltered \
-out batch_3_BowtieBlastFiltered

Use python script to filter only wanted loci:

In [47]:
cd ../scripts

/Volumes/Time Machine Backups/Cod-Time-Series-Data/scripts


In [30]:
!python checkBlastResults_DD.py \
../Blast_b3_20k/batch_3_BowtieBlastFiltered \
../Blast_b3_20k/batch_3_BOWTIEout_filtered.fa \
../Blast_b3_20k/batch_3_BowtieBlastFiltered_GOOD.fa \
../Blast_b3_20k/batch_3_BowtieBlastFiltered_BAD.fa


Identifying which loci are 'good' and 'bad' based on BLAST alignments...
Writing 'good' and 'bad' loci to their respective files...


And now to check how many loci were retained:

In [33]:
cd ../Blast_b3_20k/

/Volumes/Time Machine Backups/Cod-Time-Series-Data/Blast_b3_20k


In [34]:
!grep ">" batch_3_BowtieBlastFiltered_GOOD.fa | wc -l

   14961


In [35]:
!grep ">" batch_3_BowtieBlastFiltered_BAD.fa | wc -l

     321


Blast filtered out an additional 321, so in total the Bowtie & Blast filter steps removed 404 loci leaving a total of 14961 loci.


### 8. Create final reference genome with ``bowtie``

Lastly, I need to use Bowtie again to build a final Bowtie index using the files cleaned in Blast, and then use Bowtie to align all of my fastq files to the Bowtie index for ``pstacks``. So I manually need to move the output files from Blast to the Bowtie folder.

In [49]:
cd /Volumes/Time Machine Backups/Cod-Time-Series-Data/Bowtie/bowtie-1.1.2

/Volumes/Time Machine Backups/Cod-Time-Series-Data/Bowtie/bowtie-1.1.2


In [40]:
!./bowtie-build batch_3_BowtieBlastFiltered_GOOD.fa batch_3_final_index

Settings:
  Output files: "batch_3_final_index.*.ebwt"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 5 (one in 32)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  batch_3_BowtieBlastFiltered_GOOD.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 538596
Using parameters --bmax 403947 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 403947 --dcv 1024
Constructing suffix-a

The fastq files are in a directory within the bowtie directory, and I need to unzip them. I used the following code through a terminal outside jupyter notebook because I wasn't sure if it would crash the kernel.

```
gunzip *.gz
```

Then, I can run this python script to make the final SAM file that will feed into ``pstacks``.

In [52]:
!python final_bowtie_shell.py prt_out_filenames_20k.txt 3 batch_3_final_index batch_3_final.sam

You are working with 82 files. Consider checking your code if you expected a different number of files.
^CTraceback (most recent call last):
  File "final_bowtie_shell.py", line 51, in <module>
    subprocess.call(["sh final_bowtie_shell.txt"], shell = True) # run shell script
  File "//anaconda/lib/python2.7/subprocess.py", line 523, in call
    return Popen(*popenargs, **kwargs).wait()
  File "//anaconda/lib/python2.7/subprocess.py", line 1392, in wait
    pid, sts = _eintr_retry_call(os.waitpid, self.pid, 0)
  File "//anaconda/lib/python2.7/subprocess.py", line 476, in _eintr_retry_call
    return func(*args)
KeyboardInterrupt



### 9. ``pstacks``

``pstacks`` [documentation](http://catchenlab.life.illinois.edu/stacks/comp/pstacks.php) highlights:

pstacks -t file_type -f file_path [-o path] [-i id] [-m min_cov] [-p num_threads] [-h]
<br>t — input file Type. Supported types: bowtie, sam, or bam.
<br>f — input file path.
<br>o — output path to write results.
<br>i — SQL ID to insert into the output to identify this sample.
<br>m — minimum depth of coverage to report a stack (default 1).
<br>p — enable parallel execution with num_threads threads.
<br>h — display this help messsage.
<br>--pct_aln [num] — require read alignments to use at least this percentage of the read (default 85%).
<br>--keep_sec_alns — keep secondary alignments (default: false, only keep primary alignments).

In my custom script, arguments are as follows:
python 
<br>{0}[pypipe_pstacks.py] 
<br>{1}[number of threads]
<br>{2}[file type, either "sam" or "bowtie"]
<br>{3}[input file path]
<br>{4}[output path to write results]
<br>{5}[SQL ID number]
<br>{6}[minimum depth of coverage to report stack, default = 1]

In [44]:
pwd

u'/Volumes/Time Machine Backups/Cod-Time-Series-Data/Bowtie/bowtie-1.1.2'

In [45]:
cd ../../scripts

/Volumes/Time Machine Backups/Cod-Time-Series-Data/scripts


In [42]:
!python pypipe_pstacks.py 3 bowtie ../bowtie-1.1.2/batch_3_final. ../bowtie-1.1.2 3 3 

Min depth of coverage to report a stack: 3
Model type: SNP
Alpha significance level for model: 0.05
Error opening input file '../bowtie-1.1.2/batch_2_ReferenceGenome.'
Parsing ../bowtie-1.1.2/batch_2_ReferenceGenome.
Loading aligned sequences...done
Error: Unable to load data from '../bowtie-1.1.2/batch_2_ReferenceGenome.'.
Finished running pstacks_shell.txt script.


### 10. ``sstacks``

``sstacks`` [documentation](http://catchenlab.life.illinois.edu/stacks/comp/sstacks.php) highlights:

<br>sstacks -b batch_id -c catalog_file -s sample_file [-s sample_file_2 ...] [-o path] [-p num_threads] [-g] [-x] [-v] [-h]
<br>p — enable parallel execution with num_threads threads.
<br>b — MySQL ID of this batch.
<br>c — TSV file from which to load the catalog loci.
<br>s — TSV file from which to load sample loci.
<br>o — output path to write results.
<br>g — base matching on genomic location, not sequence identity.
<br>x — don’t verify haplotype of matching locus.
<br>v — print program version.
<br>h — display this help messsage.

Running custom python [script for ``sstacks``](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Scripts/pypipe_sstacks.py):

In [ ]:
!python pypipe_sstacks.py new_filenames_shell.txt 2 ustacks_out/batch_1 ustacks_out 10 -o sstacks_out

### 11. ``populations``

Documentation [here](http://catchenlab.life.illinois.edu/stacks/comp/populations.php).

Example code:
```
!populations -b 2 -P ustacks_out -M popmap1.txt -t 10 -r 0.50 -p 2 -m 5 --genepop
```

### 12. Additional filtering with Marine's scripts

### 13. Statistical tests in R